In [ ]:
import os
import shutil

import fiona
import geopandas as gpd
from seabeepy.config import SETTINGS
from geo.Geoserver import Geoserver

# Upload vector data to GeoServer/GeoNode

In [ ]:
# Authernticate with GeoServer
geo = Geoserver(
    "https://geonode.seabee.sigma2.no/geoserver",
    username=SETTINGS.GEOSERVER_USER,
    password=SETTINGS.GEOSERVER_PASSWORD,
)

## 1. Uploading layers from geopackages

In theory it should be possible to do this directly, but I can't get the syntax correct. For now, the code below loops over all layers in the `.gpkg`, converts them to zipped shapefiles, then uploads these to GeoServer. As with the image mosaics, it is necessary to run the `updatelayers` command - either via the GeoNode UI or the API - before the new layers will appear in the interface.

In [ ]:
temp_fold = r"/home/notebook/vecs"
gpkg_path = r"/home/notebook/shared-seabee-ns9879k/niva/2022/2022-08-31_RUNDE/Annotation/vector/2022-08-31_Remoy_Annotation.gpkg"

for layer_name in fiona.listlayers(gpkg_path):
    # Read layer
    gdf = gpd.read_file(gpkg_path, layer=layer_name)

    # Save as zipped shapefile
    shp_fold = os.path.join(temp_fold, layer_name)
    os.makedirs(shp_fold)
    shp_path = os.path.join(shp_fold, layer_name + ".shp")
    gdf.to_file(shp_path)
    zip_path = os.path.join(temp_fold, layer_name)
    shutil.make_archive(zip_path, "zip", shp_fold)

    # Add to geoserver
    geo.create_shp_datastore(
        path=zip_path + ".zip", store_name=layer_name, workspace="geonode"
    )

    # Tidy up
    os.remove(zip_path + ".zip")
    shutil.rmtree(shp_fold)